# NSRDB<a id='NSRDB'></a>

### Introduction

In this notebook, I will be interacting with the NSRDB through the use of the HSDS. The data will be used as input for the final model to calculate ROI for U.S. customers. I will be looking through this data to determine how best to use it in my final tool. The end result will be a function that can extract the necessary information from the various datasets provided by the NSRDB. I'll also create a dataframe of electricity prices by state.

### Imports

In [27]:
import warnings
warnings.simplefilter('ignore')
import h5pyd
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
import os
import matplotlib.pyplot as plt
import csv
from pandas_profiling import ProfileReport

### Load File

In [2]:
nsrdb = h5pyd.File("/nrel/nsrdb/v3/nsrdb_2020.h5", 'r')

The year 2020 is the most recent, so that is the one that we will use. Let's look at the different datasets in the file.

### Look at Data

In [3]:
dsets = pd.DataFrame(nsrdb)
dsets.rename(columns = {0:'dataset'}, inplace = True)

In [4]:
dsets

,dataset
0,air_temperature
1,alpha
2,aod
3,asymmetry
4,cld_opd_dcomp
5,cld_reff_dcomp
6,clearsky_dhi
7,clearsky_dni
8,clearsky_ghi
9,cloud_press_acha


Interesting. This is much more than we have in the UK data. And some that we have in the UK data are not present here. No bother, we'll cross that bridge when we come to it.

In [18]:
air_temperature = nsrdb['air_temperature']
alpha = nsrdb['alpha']
aod = nsrdb['aod']
asymmetry = nsrdb['asymmetry']
cld_opd_dcomp = nsrdb['cld_opd_dcomp']
cld_reff_dcomp = nsrdb['cld_reff_dcomp']
clearsky_dhi = nsrdb['clearsky_dhi']
clearsky_dni = nsrdb['clearsky_dni']
clearsky_ghi = nsrdb['clearsky_ghi']
cloud_press_acha = nsrdb['cloud_press_acha']
cloud_type = nsrdb['cloud_type']
coordinates = nsrdb['coordinates']
dew_point = nsrdb['dew_point']
dhi = nsrdb['dhi']
dni = nsrdb['dni']
ghi = nsrdb['ghi']
fill_flag = nsrdb['fill_flag']
meta = nsrdb['meta']
ozone = nsrdb['ozone']
relative_humidity = nsrdb['relative_humidity']
solar_zenith_angle = nsrdb['solar_zenith_angle']
ssa = nsrdb['ssa']
surface_albedo = nsrdb['surface_albedo']
surface_pressure = nsrdb['surface_pressure']
time_index = nsrdb['time_index']
total_precipitable_water = nsrdb['total_precipitable_water']
wind_direction = nsrdb['wind_direction']
wind_speed = nsrdb['wind_speed']

### Load Attributes

Let's load up a dataframe with all of the attributes of the various datasets. 

In [19]:
vals = []
for dset in dsets['dataset']:
    attr_series = pd.Series(nsrdb[dset].attrs)
    vals.append(attr_series)
attrs = pd.DataFrame(vals)
attrs.head()

,data_source,elevation_correction,physical_max,physical_min,psm_scale_factor,psm_units,scale_factor,spatial_interp_method,temporal_interp_method,units,description,freq,time_zone
0,MERRA2,True,70.0,-100.00,10.0,Celsius,10.0,IDW2,linear,Celsius,NaN,NaN,NaN
1,MERRA2,False,2.5,0.00,100.0,unitless,100.0,IDW2,nearest,unitless,NaN,NaN,NaN
2,MERRA2,True,1.5,0.01,10000.0,unitless,10000.0,IDW2,nearest,unitless,NaN,NaN,NaN
3,climatology,False,1.0,-1.00,100.0,unitless,100.0,NN,nearest,unitless,NaN,NaN,NaN
4,UW-GOES,False,80.0,0.00,100.0,unitless,100.0,nan,nan,unitless,NaN,NaN,NaN


In [20]:
attrs = attrs.merge(dsets, how = 'outer', left_index = True, right_index = True).T
attrs

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
data_source,MERRA2,MERRA2,MERRA2,climatology,UW-GOES,UW-GOES,output,output,output,UW-GOES,...,MERRA2,derived,calculated,MERRA2,MODIS-IMS,MERRA2,NaN,MERRA2,MERRA2,MERRA2
elevation_correction,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,NaN,True,False,False
physical_max,70.0,2.5,1.5,1.0,80.0,80.0,800.0,1350.0,1350.0,1100.0,...,0.5,100.0,180.0,1.0,1.0,1100.0,NaN,15.0,360.0,40.0
physical_min,-100.0,0.0,0.01,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2,0.0,0.0,0.0,0.0,300.0,NaN,0.0,0.0,0.0
psm_scale_factor,10.0,100.0,10000.0,100.0,100.0,100.0,1.0,1.0,1.0,1.0,...,1000.0,100.0,100.0,100.0,100.0,1.0,NaN,10.0,1.0,10.0
psm_units,Celsius,unitless,unitless,unitless,unitless,micron,W/m2,W/m2,W/m2,hPa,...,atm-cm,%,degrees,unitless,unitless,hPa,NaN,cm,degrees,m/s
scale_factor,10.0,100.0,10000.0,100.0,100.0,100.0,1.0,1.0,1.0,1.0,...,1000.0,100.0,100.0,100.0,100.0,1.0,NaN,10.0,1.0,10.0
spatial_interp_method,IDW2,IDW2,IDW2,NN,nan,nan,nan,nan,nan,nan,...,IDW2,nan,nan,NN,AGG4,IDW2,NaN,IDW2,NN,IDW4
temporal_interp_method,linear,nearest,nearest,nearest,nan,nan,nan,nan,nan,nan,...,linear,nan,nan,linear,nearest,linear,NaN,linear,linear,linear
units,Celsius,unitless,unitless,unitless,unitless,micron,W/m2,W/m2,W/m2,hPa,...,atm-cm,%,degrees,unitless,unitless,hPa,NaN,cm,degrees,m/s


Let's make the dataset row into the column names for easy lookup

In [21]:
attrs.columns = pd.Series(attrs.loc['dataset'])
attrs.drop('dataset', inplace = True)
attrs

dataset,air_temperature,alpha,aod,asymmetry,cld_opd_dcomp,cld_reff_dcomp,clearsky_dhi,clearsky_dni,clearsky_ghi,cloud_press_acha,...,ozone,relative_humidity,solar_zenith_angle,ssa,surface_albedo,surface_pressure,time_index,total_precipitable_water,wind_direction,wind_speed
data_source,MERRA2,MERRA2,MERRA2,climatology,UW-GOES,UW-GOES,output,output,output,UW-GOES,...,MERRA2,derived,calculated,MERRA2,MODIS-IMS,MERRA2,NaN,MERRA2,MERRA2,MERRA2
elevation_correction,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,NaN,True,False,False
physical_max,70.0,2.5,1.5,1.0,80.0,80.0,800.0,1350.0,1350.0,1100.0,...,0.5,100.0,180.0,1.0,1.0,1100.0,NaN,15.0,360.0,40.0
physical_min,-100.0,0.0,0.01,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.2,0.0,0.0,0.0,0.0,300.0,NaN,0.0,0.0,0.0
psm_scale_factor,10.0,100.0,10000.0,100.0,100.0,100.0,1.0,1.0,1.0,1.0,...,1000.0,100.0,100.0,100.0,100.0,1.0,NaN,10.0,1.0,10.0
psm_units,Celsius,unitless,unitless,unitless,unitless,micron,W/m2,W/m2,W/m2,hPa,...,atm-cm,%,degrees,unitless,unitless,hPa,NaN,cm,degrees,m/s
scale_factor,10.0,100.0,10000.0,100.0,100.0,100.0,1.0,1.0,1.0,1.0,...,1000.0,100.0,100.0,100.0,100.0,1.0,NaN,10.0,1.0,10.0
spatial_interp_method,IDW2,IDW2,IDW2,NN,nan,nan,nan,nan,nan,nan,...,IDW2,nan,nan,NN,AGG4,IDW2,NaN,IDW2,NN,IDW4
temporal_interp_method,linear,nearest,nearest,nearest,nan,nan,nan,nan,nan,nan,...,linear,nan,nan,linear,nearest,linear,NaN,linear,linear,linear
units,Celsius,unitless,unitless,unitless,unitless,micron,W/m2,W/m2,W/m2,hPa,...,atm-cm,%,degrees,unitless,unitless,hPa,NaN,cm,degrees,m/s


### Example Dataset

Let's look at the air temperature dataset; just the first 100. We will use the attrs dataframe to get the scale factor for this feature so that we can unscale it

In [16]:
air_temp = pd.DataFrame(air_temperature[:,:100] / attrs.loc['scale_factor', 'air_temperature'])
air_temp.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,27.5,27.6,27.6,27.8,27.8,27.8,27.8,27.8,27.8,27.8,...,26.4,26.4,26.4,26.4,26.4,26.3,26.3,26.3,26.3,26.3
1,27.5,27.6,27.6,27.7,27.8,27.8,27.8,27.8,27.8,27.8,...,26.4,26.4,26.4,26.4,26.3,26.3,26.3,26.3,26.2,26.2
2,27.5,27.6,27.6,27.7,27.7,27.8,27.8,27.8,27.8,27.7,...,26.4,26.4,26.3,26.3,26.3,26.3,26.2,26.2,26.2,26.2
3,27.4,27.5,27.5,27.7,27.7,27.7,27.7,27.7,27.7,27.7,...,26.4,26.4,26.3,26.3,26.3,26.3,26.2,26.2,26.2,26.1
4,27.4,27.5,27.5,27.6,27.6,27.7,27.7,27.7,27.7,27.6,...,26.4,26.4,26.3,26.3,26.3,26.2,26.2,26.2,26.1,26.1


The air temperature dataset has a column for each of the over 2000000 lat,lon pairs and a row for each measurement (30 minute intervals). We can use the coordinates dataframe to get the column index of a particular point. We'll look at a function to do that in a little bit, shown initially in the NSRDB example on github [link at the bottom of notebook].

In [10]:
coordinates_df = pd.DataFrame(coordinates[...])

In [11]:
coordinates_df.head()
coordinates_df.rename(columns = {0:'latitude', 1:'longitude'}, inplace = True)
coordinates_df

,latitude,longitude
0,-19.990000,-175.259995
1,-19.990000,-175.220001
2,-19.990000,-175.179993
3,-19.990000,-175.139999
4,-19.990000,-175.100006
...,...,...
2018387,51.810001,179.860001
2018388,51.849998,179.860001
2018389,51.689999,179.860001
2018390,51.770000,179.860001


The meta dataset contains population, location, and elevation data about each location

In [12]:
meta_df = pd.DataFrame(meta[...])
meta_df.head()

,latitude,longitude,elevation,timezone,country,state,county,urban,population,landcover
0,-19.99,-175.259995,0.0,13,b'None',b'None',b'None',b'None',-9999,210
1,-19.99,-175.220001,0.0,13,b'None',b'None',b'None',b'None',-9999,210
2,-19.99,-175.179993,0.0,13,b'None',b'None',b'None',b'None',-9999,210
3,-19.99,-175.139999,0.0,13,b'None',b'None',b'None',b'None',-9999,210
4,-19.99,-175.100006,0.0,13,b'None',b'None',b'None',b'None',-9999,210


Let's look at one state, for instance. I just visited Colorado.

### State Info

In [13]:
CO = meta_df.loc[meta_df['state'] == b'Colorado']

In [14]:
CO.head()

,latitude,longitude,elevation,timezone,country,state,county,urban,population,landcover
114344,37.009998,-109.019997,1437.040039,-7,b'United States',b'Colorado',b'Montezuma',b'None',112,130
114345,37.009998,-108.980003,1469.000000,-7,b'United States',b'Colorado',b'Montezuma',b'None',21,130
114346,37.009998,-108.940002,1455.400024,-7,b'United States',b'Colorado',b'Montezuma',b'None',1,130
114347,37.009998,-108.900002,1470.250000,-7,b'United States',b'Colorado',b'Montezuma',b'None',0,130
114348,37.009998,-108.860001,1489.439941,-7,b'United States',b'Colorado',b'Montezuma',b'None',0,130


In [15]:
CO.shape

(17500, 10)

All of the text columns are byte objects (b in front of the text). We can use decode to fix that.

In [16]:
states_encoded = meta_df[meta_df['country'] == b'United States']['state'].unique()

In [17]:
states = []
for state in states_encoded:
    states.append(state.decode("utf-8"))
states = pd.DataFrame(states, columns = ['State'])
print(states.shape)
states.head()

(51, 1)


,State
0,Hawaii
1,Texas
2,Arizona
3,New Mexico
4,California


### Electricity Prices

Now that we have a states dataframe, this would be a good time to enter all of those electricity prices. Let's do that and give them their own column. The information comes from the U.S. Energy Information Administration, https://www.eia.gov/electricity/state/. The prices are entered according to a alphabetic ordering of states, so we will need to sort the 'State' columns of the states dataframe before added the electricity prices.

In [18]:
electricity_prices = [9.84, 19.82, 10.44, 8.32, 18.00, 10.27, 19.13, 10.24, 11.90, 10.06, 9.93, 27.55, 7.99, 9.75, 9.92, 8.97, 10.38, 8.58, 7.51, 13.54, 11.15, 18.19, 12.21, 10.57, 9.13, 9.64, 9.13, 8.97, 8.33, 16.63, 13.63, 9.33, 14.87, 9.43, 8.53, 9.44, 7.63, 8.82, 9.7, 18.54, 9.9, 10.06, 9.52, 8.36, 8.27, 16.33, 9.16, 8.33, 8.75, 10.82, 8.27]
states.sort_values('State', ascending = True, ignore_index = True, inplace = True)
states['Electricity Prices'] = electricity_prices
states.head()

,State,Electricity Prices
0,Alabama,9.84
1,Alaska,19.82
2,Arizona,10.44
3,Arkansas,8.32
4,California,18.00


Here is that function that I mentioned. This will allow us to find the nearest point, to a point entered by a user, that we have data for. It used the CKDTree package and the query method. The function was pulled from the NSRDB github, link below. I will be using a random point in Colorado as "myHouse". Note, this is not my actual location.

### Find Nearest Index

In [19]:
dset_coords = coordinates
tree = cKDTree(dset_coords)
def nearest_site(tree, lat_coord, lon_coord):
    lat_lon = np.array([lat_coord, lon_coord])
    dist, pos = tree.query(lat_lon)
    return pos

myHouse = (40.072816, -105.448832)
myHouse_idx = nearest_site(tree, myHouse[0], myHouse[1] )
myHouse_idx

150163

Let's use that index to look up some information on that point

In [20]:
myHouse_df = pd.DataFrame(air_temperature[:, myHouse_idx]  / attrs.loc['scale_factor', 'air_temperature'] , columns = ['Air Temp'])
myHouse_state = meta_df.iloc[myHouse_idx]
myHouse_state

latitude                 40.09
longitude          -105.459999
elevation          2677.840088
timezone                    -7
country       b'United States'
state              b'Colorado'
county              b'Boulder'
urban                  b'None'
population                 124
landcover                   70
Name: 150163, dtype: object

In [21]:
myHouse_df.head()

,Air Temp
0,-7.5
1,-7.7
2,-7.8
3,-7.8
4,-7.9


In [22]:
myHouse_df['Dew Point'] = pd.Series(dew_point[:, myHouse_idx] / attrs.loc['scale_factor', 'dew_point'])
myHouse_df.head()

,Air Temp,Dew Point
0,-7.5,-12.7
1,-7.7,-12.7
2,-7.8,-12.9
3,-7.8,-12.9
4,-7.9,-13.1


### Column Extraction

This is how we will do it in practice. We take in an latitude and longitude from the user. Then, we will grow a dataframe with all of the required information, such that it can be passed to the model for processing. After passing that dataframe to the model, we will return to the user a summary of statistics about their upcoming solar project [how much they will save per year, peak times, etc]. The function below will handle just the building of the dataframe. A later notebook will deal with summary statistics and address to lat,lon conversion.

In [23]:
def getLocationData(lat, lon, cols = ['meta']):
    
    tree = cKDTree(nsrdb['coordinates'])
    dist, pos = tree.query(np.array([lat, lon]))
    
    df = pd.DataFrame(columns = cols)
    
    for col in cols:
        df[col] = nsrdb[col][:, pos] / attrs.loc['scale_factor', col]
    
    return df
    #do something else

In [25]:
MyHouse = getLocationData(40.072816, -105.448832, cols = ['air_temperature', 'dew_point', 'surface_pressure'])

In [26]:
MyHouse.head()

,air_temperature,dew_point,surface_pressure
0,-7.5,-12.7,743.0
1,-7.7,-12.7,743.0
2,-7.8,-12.9,743.0
3,-7.8,-12.9,742.0
4,-7.9,-13.1,742.0


Looks like the un-scaling worked for all of the features. Let's generate a profile report to really dive into this data. [The report will not show up on github. Save the file and run it yourself to see the report.]

In [27]:
profile = ProfileReport(MyHouse, title="Profiling Report for Sample NSRDB Dataframe")
profile.to_widgets()

Summarize dataset:   0%|          | 0/17 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Woohoo! That looks great. We were able to choose a location and and which columns we wanted and got a nice dataframe in return. And with no missing data, that's wonderful! When we finish training our model, we'll know which columns we need. This function is nice because it allows the inputs of the model to change while this function can remain the same, giving whichever columns are asked for instead of a set few

Let's save our electricity prices dataframe and our attributes dataframe to csv files

### Save Data

In [28]:
states.to_csv('../data/StatesElectricity.csv')
attrs.to_csv('../data/Attributes.csv')

### Citations

Sengupta, M., Y. Xie, A. Lopez, A. Habte, G. Maclaurin, and J. Shelby. 2018. "The National Solar Radiation Data Base (NSRDB)." Renewable and Sustainable Energy Reviews 89 (June): 51-60

NREL, hsds-examples, 03_NSRDB_introduction.ipynb, Github. November 1, 2022. https://github.com/NREL/hsds-examples/blob/master/notebooks/03_NSRDB_introduction.ipynb